In [2]:
import pdfplumber
import pandas as pd
import re
from pathlib import Path
from tqdm import tqdm

In [2]:
def standardize_name(name):
    if ',' in name:
        last, first = name.split(',', 1)
        return f"{first.strip()} {last.strip()}"
    return name.strip()

def standardize_birthyear(year):
    if pd.isna(year):
        return None
    
    year = str(year).strip()  # in String umwandeln
    
    # Sonderfall: '0' → 2000
    if year == '0':
        return 2000
    
    # Vierstelliges Jahr, z.B. '2014'
    if len(year) == 4:
        return int(year)
    
    # Zweistelliges Jahr, z.B. '80', '98', '00', '01'
    if len(year) == 2:
        yy = int(year)
        if yy >= 20:        # alles >=50 → 1900er
            return 1900 + yy
        else:               # alles <50 → 2000er
            return 2000 + yy
    
    # Einstellige Zahl, z.B. '9' → 2009?
    if len(year) == 1:
        return 2000 + int(year)
    
    # Alles andere: zurückgeben als int, falls möglich
    try:
        return int(year)
    except:
        return None

In [ ]:


# Folder with PDFs
# pdf_folder = Path("data01-17")
# pdf_folder = Path("data24")
# pdf_folder = Path("data18-22")

# Regex for result lines (Platz wird ignoriert)
# 01-17


# 01-17
line_pattern_01_17 = re.compile(
    r"^(?:([1-9]\d?|50)\s+)?"                        # optional Platz
    r"(?P<leistung>("
    r"\d{1,2},\d{2}|"                               # SS,SS
    r"\d{1,2}:\d{2},\d{2}|"                         # M:SS,SS
    r"\d{1,2}:\d{2}:\d{2}|"                         # H:MM:SS
    r"\d{1,2}:\d{2}|"                               # M:SS
    r"\d{1,3}(?:\.\d{3})?"                          # Punkte, z.B. 8307
    r"))\s*"                                        # Leistung
    r"(?:\(?\s*(?P<wind>[+-]?\d,\d)\s*\)?\s+)?"     # optionaler Wind mit oder ohne ()
    r"(?P<name>[A-Za-zÄÖÜäöüß ,\-]+?)\s+"           # Name
    r"(?P<geburtsjahr>\d{2,4})\s+"                  # Geburtsjahr
    r"(?P<verein>[A-Za-zÄÖÜäöüß0-9 .\-()/]+)\s+"    # Verein
    r"(?P<datum>\d{2}\.\d{2}\.(?:\d{2,4})?)\s+"     # Datum
    r"(?P<ort>[A-Za-zÄÖÜäöüß .\-()/]+)$"            # Ort
)

# 24
line_pattern_24 = re.compile(
    r"^(?:[1-9]\d?|50)\s+"                              # Platz 1–50, zwingend
    r"(?P<leistung>\d{1,2},\d{2})\s+"                  # Leistung
    r"(?:(?P<wind>[+-]?\d,\d)\s+)?"                    # optionaler Wind
    r"(?P<name>[A-Za-zÄÖÜäöüß'´`\- ]+?)\s+"            # Name
    r"(?P<geburtsjahr>\d{4})"                          # Geburtsjahr
    r"(?P<verein>[A-Za-zÄÖÜäöüß0-9 .\-()/]+?)\s+"      # Verein, direkt danach möglich
    r"(?P<datum>\d{2}\.\d{2}\.\d{4})\s+"               # Datum
    r"(?P<ort>[A-Za-zÄÖÜäöüß .\-()]+)$"                # Ort
)

# 23
line_pattern_23 = re.compile(
    r"^(?:([1-9]\d?|50)\s+)?"                        # Platz optional (1–50)
    r"(?P<leistung>\d{1,2},\d{2})\s+"               # Leistung
    r"(?:(?P<wind>\(?[+-]?\d,\d\)?)\s+)?"           # optionaler Wind, auch in Klammern
    r"(?P<name>[A-Za-zÄÖÜäöüß'´`\- ]+?)\s+"         # Name
    r"(?P<geburtsjahr>\d{4})\s+"                    # Geburtsjahr
    r"(?P<verein>[A-Za-zÄÖÜäöüß0-9 .\-()/]+?)\s+"   # Verein
    r"(?P<datum>\d{2}\.\d{2}\.\d{4})\s+"            # Datum
    r"(?P<ort>[A-Za-zÄÖÜäöüß .\-()/]+)$"            # Ort
)

# 18-22
line_pattern_18_22 = re.compile(
    r"^(?:([1-9]\d?|50)\s+)?"                              # Platz optional
    r"(?P<leistung>("
        r"\d{1,2},\d{2}|"                                 # SS,SS
        r"\d{1,2}:\d{2},\d{2}|"                           # M:SS,SS
        r"\d{1,2}:\d{2}:\d{2}|"                           # H:MM:SS
        r"\d{1,2}:\d{2}|"                                 # M:SS
        r"\d{1,3}(?:\.\d{3})?"                            # Punkte, z.B. 8307
    r"))\s*"
    r"(?:(?:\(?\s*(?P<wind>[+-]?\d,[0-9])\s*\)?)\s+)?"     # Wind optional, auch mit ()
    r"(?P<name>[A-Za-zÄÖÜäöüß'´`\- ]+?)\s+"                # Name
    r"(?P<geburtsjahr>\d{2,4})\s+"                         # Geburtsjahr
    r"(?P<verein>[A-Za-zÄÖÜäöüß0-9 .\-()/]+?)\s+"          # Verein
    r"(?P<datum>\d{2}\.\d{2}(?:\.\d{2,4})?)\.?\s+"         # HIER IST DIE KORREKTUR: \.? hinzugefügt
    r"(?P<ort>[A-Za-zÄÖÜäöüß .\-()/]+)$",                  # Ort
    re.UNICODE
)

pdf_year = {
    "data01-17": line_pattern_01_17,
    "data18-22": line_pattern_18_22,
    "data24": line_pattern_24,
    "data23": line_pattern_23
}

# Laufdisziplinen priorisiert (lange zuerst)
lauf_pattern = (
    r"10\s?km Straßengehen|20\s?km Straßengehen|50\s?km Straßengehen|"
    r"(?:10|20|50)\s?km\s?Gehen|(?:10|20|50)\s?k\s?Gehen|"
    r"(?:5|10|20|50|100)\s?km|"
    r"(?:1\.?000|1\.?500|2\.?000|3\.?000|5\.?000|10\.?000)\s?m\s?(?:Bahngehen|Gehen|Hindernis|Hürden)?|"
    r"(?:60|80|100|110|200|300|400|800|1000|1500|2000|3000|5000|10000)\s?m\s?(?:Hindernis|Hürden)?|"
    r"^(60|80|100|110|200|300|400|800|1000|1500|2000|3000|5000|10000)\s*[\u00A0\u202F]?\s*m\b"
    r"Halbmarathon|Marathon"
)

lauf_regex = re.compile(lauf_pattern, re.IGNORECASE)


# Komplette Disziplin-Regex
discipline_pattern = re.compile(
    r"^("
    + lauf_pattern + "|" +
    r"Weitsprung|Hochsprung|Dreisprung|Stabhochsprung|"
    r"Kugelstoß|Speerwurf|Diskuswurf|Hammerwurf|"
    r"Zehnkampf|Siebenkampf|Fünfkampf|10-Kampf|7-Kampf|5-Kampf"
    r")",
    re.IGNORECASE
)

discipline_standardization = {
    # Läufe
    "60m": "60 m", "100m": "100 m", "200m": "200 m", "300m": "300 m",
    "400m": "400 m", "800m": "800 m", "1000m": "1000 m", "1500m": "1500 m",
    "3000m": "3000 m", "5000m": "5000 m", "10000m": "10 000 m",
    "5km": "5 km", "10km": "10 km", "20km": "20 km",
    "10 km Straßengehen": "10 km Gehen",
    "10 km Gehen": "10 km Gehen",
    "20 km Gehen": "20 km Gehen",
    "20 km Straßengehen": "20 km Gehen",
    "5000 m Bahngehen": "5000 m Gehen",

    # Hürden
    "100 m Hürden": "100 m Hürden", "110 m Hürden": "110 m Hürden", "400 m Hürden": "400 m Hürden",

    # Sprung / Wurf
    "Weitsprung": "Weitsprung", "Hochsprung": "Hochsprung", "Dreisprung": "Dreisprung",
    "Stabhochsprung": "Stabhochsprung", "Kugelstoß": "Kugelstoß", "Speerwurf": "Speerwurf",
    "Diskuswurf": "Diskuswurf", "Hammerwurf": "Hammerwurf",

    # Mehrkampf
    "Zehnkampf": "Zehnkampf", "Siebenkampf": "Siebenkampf"
}

# Funktion zum Parsen von Dateinamen
def parse_filename(filename):
    stem = Path(filename).stem

    # Jahr
    year_match = re.search(r"bestenliste(\d{4})", stem)
    year = year_match.group(1) if year_match else ""

    # Geschlecht
    if any(s in stem for s in ["maennliche", "maenner", "junioren"]):
        gender = "M"
    elif any(s in stem for s in ["weibliche", "frauen", "juniorinnen"]):
        gender = "W"
    else:
        gender = ""

    # Altersklasse
    if "maenner" in stem:
        age_class = "Männer"
    elif "frauen" in stem:
        age_class = "Frauen"
    elif "junioren" in stem or "juniorinnen" in stem:
        age_class = "U23"
    else:
        u_match = re.search(r"U\d{2}$", stem)
        if u_match:
            age_class = u_match.group(0)
        else:
            num_match = re.search(r"(\d{1,2})$", stem)
            age_class = num_match.group(1) if num_match else ""

    return year, gender, age_class

# Funktion zum Ersetzen von Umlauten
def replace_umlauts(text):
    if not isinstance(text, str):
        return text
    replacements = {
        "ä": "ae", "ö": "oe", "ü": "ue",
        "Ä": "Ae", "Ö": "Oe", "Ü": "Ue",
        "ß": "ss"
    }
    for orig, repl in replacements.items():
        text = text.replace(orig, repl)
    return text

# Parse PDF
def parse_pdf(pdf_path):
    
    year, gender, age_class = parse_filename(pdf_path.name)

    results = []
    current_discipline = None
    ignore_section = False
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue
            for line in text.split("\n"):
                line = line.strip()
                if not line:
                    continue
                if line.startswith("Ausländer"):
                    ignore_section = True
                    continue
                if re.search(r"(?i)(mannschaft|staffel|\d+\s?x\s?\d+\s?m)", line):
                    ignore_section = True
                    continue
                
                # Wenn wir im Ausländer-Abschnitt sind, bis zur nächsten Disziplin ignorieren
                if ignore_section:
                    # Disziplin erkannt → Ausländer-Abschnitt endet
                    if discipline_pattern.match(line):
                        ignore_section = False
                    else:
                        continue
                # Skip team results
                if "Mannschaft" in line or "Mannschaftswertung" in line:
                    continue
                # Skip lines in parentheses (individuals within team)
                if line.startswith("(") and line.endswith(")"):
                    continue
                # Skip Staffel lines
                if re.search(r"\dx\d+", line):
                    ignore_section = True
                    current_discipline = None
                    continue
                # Discipline detected
                match_d = discipline_pattern.match(line)
                if match_d:
                    current_discipline = match_d.group(0).strip()
                    # Standardisierte Schreibweise (immer Leerzeichen)
                    current_discipline = discipline_standardization.get(current_discipline, current_discipline)
                    continue
                # Result line detected
                match = line_pattern.match(line)

                if match and current_discipline:
                    data = match.groupdict()
                    data['name'] = standardize_name(data['name'])
                    data['geburtsjahr'] = standardize_birthyear(data['geburtsjahr'])
                    data.update({
                        "jahr": year,
                        "geschlecht": gender,
                        "altersklasse": age_class,
                        "disziplin": current_discipline
                    })
                    results.append(data)
    return results


# Alle PDFs durchgehen
# all_results = []
# idx = 0
# pdf_folder = Path("data18-22")
# line_pattern = line_pattern_18_22
# for pdf_file in pdf_folder.glob("*.pdf"):
#     if "2020" not in pdf_file.name:
#         continue
#     print(f"Processing: {pdf_file.name}")
#     all_results.extend(parse_pdf(pdf_file))
#     idx += 1
#     if idx >= 10:
#         break

all_results = []
for key in pdf_year:
    pdf_folder = Path(key)
    line_pattern = pdf_year[key]
    pdf_files = list(pdf_folder.glob("*.pdf"))
    
    print(f"\n📂 Verarbeite Ordner: {pdf_folder} ({len(pdf_files)} Dateien)")
    
    for pdf_file in tqdm(pdf_files, desc=f"{key}", unit="pdf"):
        all_results.extend(parse_pdf(pdf_file))


# DataFrame
df = pd.DataFrame(all_results)

# Umlaut-/Sonderzeichen konvertieren
df = df.applymap(replace_umlauts)

print(df)

# Spalten sortieren
df = df[["jahr", "geschlecht", "altersklasse", "disziplin",
         "leistung", "wind", "name", "geburtsjahr",
         "verein", "datum", "ort"]]


# # CSV speichern
df.to_csv("Data.csv", index=False, sep=";")

print("✅ Fertig! Daten gespeichert in: Data.csv")
# df.head(10)
df



📂 Verarbeite Ordner: data01-17 (204 Dateien)


data01-17:   0%|          | 0/204 [00:00<?, ?pdf/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
data01-17: 100%|██████████| 204/204 [06:32<00:00,  1.92s/pdf]



📂 Verarbeite Ordner: data18-22 (60 Dateien)


data18-22: 100%|██████████| 60/60 [04:22<00:00,  4.37s/pdf]



📂 Verarbeite Ordner: data24 (12 Dateien)


data24: 100%|██████████| 12/12 [00:28<00:00,  2.36s/pdf]



📂 Verarbeite Ordner: data23 (12 Dateien)


data23: 100%|██████████| 12/12 [00:25<00:00,  2.15s/pdf]
/var/folders/lf/d2nc392142b837mf8sypzlx00000gn/T/ipykernel_29017/224898827.py:304: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(replace_umlauts)


       leistung  wind                name  geburtsjahr               verein  \
0         10,95  +1,3  Gina Lueckenkemper         1996  LG Olympia Dortmund   
1         11,06  +1,8       Rebekka Haase         1993     LV 90 Erzgebirge   
2         11,14  -0,2          Lisa Mayer         1996   Sprintteam Wetzlar   
3         11,24  +0,3       Tatjana Pinto         1992         LC Paderborn   
4         11,25  +1,9          Sina Mayer         1995     LAZ Zweibruecken   
...         ...   ...                 ...          ...                  ...   
222254    40,64  None       Tizian Zoeger         2010     LAC Aschersleben   
222255    40,48  None       Phil Matthias         2009             SV Halle   
222256    40,38  None        Laurin Steep         2009       MTV Dannenberg   
222257    40,36  None          Rico Lange         2009  TSV Chemie Premnitz   
222258    40,30  None     Johann Fichtner         2009   SV Preussen Berlin   

             datum                       ort  jahr 

,jahr,geschlecht,altersklasse,disziplin,leistung,wind,name,geburtsjahr,verein,datum,ort
0,2017,W,Frauen,100 m,"10,95","+1,3",Gina Lueckenkemper,1996,LG Olympia Dortmund,05.08.,London/GBR
1,2017,W,Frauen,100 m,"11,06","+1,8",Rebekka Haase,1993,LV 90 Erzgebirge,25.05.,Zeulenroda
2,2017,W,Frauen,100 m,"11,14","-0,2",Lisa Mayer,1996,Sprintteam Wetzlar,27.08.,Berlin
3,2017,W,Frauen,100 m,"11,24","+0,3",Tatjana Pinto,1992,LC Paderborn,15.07.,Ninove/BEL
4,2017,W,Frauen,100 m,"11,25","+1,9",Sina Mayer,1995,LAZ Zweibruecken,11.06.,Regensburg
...,...,...,...,...,...,...,...,...,...,...,...
222254,2023,M,14,Speerwurf,"40,64",None,Tizian Zoeger,2010,LAC Aschersleben,29.04.2023,Schoenebeck
222255,2023,M,14,Speerwurf,"40,48",None,Phil Matthias,2009,SV Halle,07.05.2023,Halle (Saale)
222256,2023,M,14,Speerwurf,"40,38",None,Laurin Steep,2009,MTV Dannenberg,16.09.2023,Fallersleben (Wolfsburg)
222257,2023,M,14,Speerwurf,"40,36",None,Rico Lange,2009,TSV Chemie Premnitz,02.09.2023,Berlin-Lichterfelde


In [3]:
df = pd.read_csv("Data.csv", sep=";")
sorted_years = sorted(df['jahr'].unique())
sorted_years

[np.int64(2001),
 np.int64(2002),
 np.int64(2003),
 np.int64(2004),
 np.int64(2005),
 np.int64(2006),
 np.int64(2007),
 np.int64(2008),
 np.int64(2009),
 np.int64(2010),
 np.int64(2011),
 np.int64(2012),
 np.int64(2013),
 np.int64(2014),
 np.int64(2015),
 np.int64(2016),
 np.int64(2017),
 np.int64(2018),
 np.int64(2019),
 np.int64(2020),
 np.int64(2021),
 np.int64(2022),
 np.int64(2023),
 np.int64(2024)]

In [11]:
df.describe(include='all')

,jahr,geschlecht,altersklasse,disziplin,leistung,wind,name,geburtsjahr,verein,datum,ort
count,222251.000000,222251,222251,222251,222251,51930,222251,222251.000000,222251,222251,222251
unique,NaN,2,7,55,43394,144,34429,NaN,5222,1575,4177
top,NaN,M,20,Hochsprung,"1,72","0,0",Carolin Schaefer,NaN,TSV Bayer Leverkusen,15.06.,Berlin
freq,NaN,111233,45405,13057,596,6723,180,NaN,3585,1836,8151
mean,2012.609685,NaN,NaN,NaN,NaN,NaN,NaN,1993.644276,NaN,NaN,NaN
std,6.844782,NaN,NaN,NaN,NaN,NaN,NaN,9.293718,NaN,NaN,NaN
min,2001.000000,NaN,NaN,NaN,NaN,NaN,NaN,1929.000000,NaN,NaN,NaN
25%,2007.000000,NaN,NaN,NaN,NaN,NaN,NaN,1988.000000,NaN,NaN,NaN
50%,2013.000000,NaN,NaN,NaN,NaN,NaN,NaN,1994.000000,NaN,NaN,NaN
75%,2019.000000,NaN,NaN,NaN,NaN,NaN,NaN,2001.000000,NaN,NaN,NaN


In [8]:
print("\n### Eindeutige Werte in kategorischen Spalten ###")

print("\n--- Geschlecht ---")
print(df['geschlecht'].unique())

print("\n--- Altersklasse ---")
# Sortieren hilft, den Bereich zu sehen
print(sorted(df['altersklasse'].unique()))

print("\n--- Disziplin ---")
print(df['disziplin'].unique())


### Eindeutige Werte in kategorischen Spalten ###

--- Geschlecht ---
['W' 'M']

--- Altersklasse ---
['14', '16', '18', '20', 'Frauen', 'Maenner', 'U23']

--- Disziplin ---
['100 m' '200 m' '400 m' '800 m' '1500 m' '3000 m' '5000 m' '10 km'
 'Marathon' '100 km' '100 m Huerden' '400 m Huerden' '3000 m Hindernis'
 'Hochsprung' 'Stabhochsprung' 'Weitsprung' 'Dreisprung' 'Kugelstoss'
 'Diskuswurf' 'Hammerwurf' 'Speerwurf' '5000 m Gehen' '10 km Gehen'
 '20 km Gehen' '110 m Huerden' '2000 m Hindernis' '300 m' '1000 m' '5 km'
 '80 m Huerden' '300 m Huerden' '3000 m Bahngehen' '50 km Gehen' '2000 m'
 '10000 m Bahngehen' '50 km Strassengehen' '1500 m Hindernis' 'Zehnkampf'
 '10000 m Gehen' '10000 m' '1.500 m' '3.000 m' '5.000 m' '10.000 m'
 '3.000 m Hindernis' 'Siebenkampf' '5.000 m Bahngehen' '1.500 m Hindernis'
 '3.000 m Bahngehen' '1.000 m' 'Fuenfkampf' '10.000 m Bahngehen'
 '2.000 m Hindernis' '2.000 m' '5.000 M']


In [44]:
csv_files = ["list_01_17.csv", "list_18_22.csv", "list_23.csv", "list_24.csv"]

dfs = [pd.read_csv(f, sep=";") for f in csv_files]
df_gesamt = pd.concat(dfs, ignore_index=True)
df_gesamt.to_csv("list01-24_all.csv", index=False, sep=";")